In [ ]:
from Preprocessing.Preprocessor import Preprocessor
from models.generic_model import Model
from data_utils.keras_generator import Generator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from utils.losses import weighted_cross_entropy_loss, f1_score_metric
from utils.callbacks import get_tensorboard_callback
from keras import optimizers
import os
import pandas as pd
import numpy as np


In [ ]:
label_encoder_dir = "Preprocessing"
val_data_dir = r'D:\Val_Data'


In [ ]:
val_files = os.listdir(val_data_dir)
array = pd.read_hdf(os.path.join(val_data_dir, val_files[0]))
P = Preprocessor(label_encoder_dir)
array = P.process_batch(array)
action_col = array.columns == 'action'
g_config = {'labels': 6,
            'weightage': [1, 1, 1, 1, 1, 1],
            'data_directory': val_data_dir,
            'file_batch_size': 50,
            'format_': 'h5',
            'ex_batch_size': 10000,
            'file_examples': 5000,
            'max_queue_size': 1,
            'Preprocessor': P,
            'static_weightage': False,
            'action_col': action_col}

G = Generator(config=g_config)


In [ ]:
nn_config = {'layer_shapes': [500, 500, 500], 'Activation': 'relu', 'Output': 6,
             'Input_shape': 472, 'model_type': 'fc'}
my_model = Model(nn_config) 
f1 = f1_score_metric(6)
my_model.model.compile(loss=weighted_cross_entropy_loss, optimizer=optimizers.Adam(lr=5e-4), metrics=["acc", f1])


In [ ]:
G.fill_buffer()


In [ ]:
my_model.model.load_weights(r'E:\Weights.h5')


In [ ]:
true_actions = []
pred_actions = []
for i in range(len(G)):
    data, actions = G[i]
    true_actions.extend(np.argmax(actions, axis=1))
    pred_actions.extend(np.argmax(my_model.model.predict(data), axis=1))
    
confusion_matrix(true_actions, pred_actions)


In [ ]:
print(f1_score(true_actions, pred_actions, average='micro'))
print(f1_score(true_actions, pred_actions, average='macro'))
print(f1_score(true_actions, pred_actions, average='weighted'))


In [ ]:
np.unique(true_actions,return_counts=True)


In [ ]:
my_model.model.fit_generator(generator=G, steps_per_epoch=len(G), epochs=10, verbose=1, max_queue_size=3)
